# Neural Network 1: XOR

In [52]:
import tensorflow as tf 
import numpy as np

## XOR with Logistic Regression

In [26]:
xy = np.loadtxt('07train.txt', unpack= True)
x_data = xy[0:-1]
y_data = xy[-1]

In [27]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W = tf.Variable(tf.random_uniform([1,len(x_data)],-1.0,1.0))

In [28]:
# hypothesis
h = tf.matmul(W,X)
hypothesis = tf.div(1.,1.+tf.exp(-h))

#cost function

cost = -tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y)*tf.log(1-hypothesis))

In [29]:
# minimize 
a = tf.Variable(0.01) # learning rate
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

In [30]:
init = tf.initialize_all_variables()

In [31]:
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(1000):
        sess.run(train,feed_dict={X:x_data,Y:y_data})
        if step % 200 == 0:
            print(step,sess.run(cost, feed_dict={X:x_data,Y:y_data}), sess.run(W))
    #test model
    correct_prediction = tf.equal(tf.floor(hypothesis+0.5), Y)
    
    # Calculate accuracy
    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))
    print(sess.run([hypothesis, tf.floor(hypothesis+0.5), correct_prediction, accuracy],feed_dict={X:x_data, Y:y_data}))
    print("Accuracy :",accuracy.eval({X:x_data,Y:y_data}))

0 0.730594 [[ 0.89144653 -0.3110007 ]]
200 0.718176 [[ 0.73556656 -0.33078527]]
400 0.710574 [[ 0.61259198 -0.33163002]]
600 0.705715 [[ 0.51485693 -0.32041195]]
800 0.702453 [[ 0.43636063 -0.30205098]]
[array([[ 0.49999809,  0.43044424,  0.59214967,  0.52318865]], dtype=float32), array([[ 0.,  0.,  1.,  1.]], dtype=float32), array([[ True, False,  True, False]], dtype=bool), 0.5]
Accuracy : 0.5


## XOR with NN

In [54]:
xy = np.loadtxt('07train.txt', unpack= True)
x_data = np.transpose(xy[0:-1]) # 그냥 T로 해도 무관하다.
y_data = np.reshape(xy[-1],(4,1)) # T로 해도 무관하다.

print(x_data)
print(y_data)

[[ 0.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  1.]]
[[ 0.]
 [ 1.]
 [ 1.]
 [ 0.]]


In [44]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2,2],-1.0,1.0)) # W1 은 2 x 2 
W2 = tf.Variable(tf.random_uniform([2,1],-1.0,1.0)) # W2 는 2 X 1

b1 = tf.Variable(tf.zeros([2]), name="Bias1")
b2 = tf.Variable(tf.zeros([1]), name="Bias2")

L2 = tf.sigmoid(tf.matmul(X,W1) + b1)
hypothesis = tf.sigmoid(tf.matmul(L2,W2) + b2)

In [56]:
# Cost function 
cost = -tf.reduce_mean( Y*tf.log(hypothesis)+(1-Y)* tf.log(1.-hypothesis) )

# Minimize cost.
a = tf.Variable(0.1)
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(8001):
        sess.run(train, feed_dict={X:x_data, Y:y_data})
        
        if step % 1000 == 0:
            print(
                step, 
                sess.run(cost, feed_dict={X:x_data, Y:y_data}), 
                sess.run(W1),
                sess.run(W2)
            )
    
    # Test model
    correct_prediction = tf.equal( tf.floor(hypothesis+0.5), Y)
    accuracy = tf.reduce_mean(tf.cast( correct_prediction, "float" ) )
    
    # Check accuracy
    print( sess.run( [hypothesis, tf.floor(hypothesis+0.5), correct_prediction, accuracy], 
                   feed_dict={X:x_data, Y:y_data}) )
    print( "Accuracy:", accuracy.eval({X:x_data, Y:y_data}) )

0 0.706673 [[-0.57628304 -0.80752927]
 [ 0.78786755  0.6393373 ]] [[ 0.11008612]
 [-0.78606141]]
1000 0.690113 [[-0.69285113 -0.67859644]
 [ 0.72962332  0.90172321]] [[ 0.37175646]
 [-0.6819883 ]]
2000 0.659352 [[-1.16083717 -1.2792629 ]
 [ 0.91882885  1.68050694]] [[ 1.01787519]
 [-1.27146041]]
3000 0.311704 [[-3.17863941 -3.33933496]
 [ 2.88076091  3.60145283]] [[ 3.87371731]
 [-3.6566453 ]]
4000 0.101457 [[-4.5485034  -4.61063957]
 [ 4.27897787  4.81617212]] [[ 6.37586975]
 [-5.86819315]]
5000 0.0538696 [[-5.1095252  -5.15615463]
 [ 4.84866285  5.35889626]] [[ 7.61145735]
 [-7.05345154]]
6000 0.0357304 [[-5.43099403 -5.47353029]
 [ 5.17402887  5.67877817]] [[ 8.38922501]
 [-7.81793737]]
7000 0.0264803 [[-5.64981937 -5.69099188]
 [ 5.39496326  5.8991189 ]] [[ 8.95111656]
 [-8.37567997]]
8000 0.020941 [[-5.81335258 -5.8540659 ]
 [ 5.55976439  6.06480885]] [[ 9.38939571]
 [-8.81286049]]
[array([[ 0.02035927],
       [ 0.98133063],
       [ 0.97328591],
       [ 0.01712324]], dtype=floa

## Use 'wide' neural network to solve XOR problem.
 - 위의 식은 1개의 layer에 2개의 노드만 존재.
 - 아래 식은 10개의 노드를 생성

In [57]:
xy = np.loadtxt('07train.txt', unpack= True)
x_data = np.transpose( xy[0:-1] )
y_data = np.reshape( xy[-1], (4,1) )

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# Wide network: Use more neurons in each layer. 
W1 = tf.Variable(tf.random_uniform( [2,10], -1.0, 1.0))
W2 = tf.Variable(tf.random_uniform( [10,1], -1.0, 1.0))

b1 = tf.Variable(tf.zeros([10]), name="Bias1")
b2 = tf.Variable(tf.zeros([1]), name="Bias2")

# Hypotheses 
L2 =  tf.sigmoid(tf.matmul(X,W1)+b1)
hypothesis = tf.sigmoid( tf.matmul(L2,W2) + b2)

# Cost function 
cost = -tf.reduce_mean( Y*tf.log(hypothesis)+(1-Y)* tf.log(1.-hypothesis) )

# Minimize cost.
a = tf.Variable(0.1)
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

# Initializa all variables.
init = tf.initialize_all_variables()


# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(8001):
        sess.run(train, feed_dict={X:x_data, Y:y_data})
        
        if step % 1000 == 0:
            print(
                step, 
                sess.run(cost, feed_dict={X:x_data, Y:y_data}), 
                sess.run(W1),
                sess.run(W2)
            )
    
    # Test model
    correct_prediction = tf.equal( tf.floor(hypothesis+0.5), Y)
    accuracy = tf.reduce_mean(tf.cast( correct_prediction, "float" ) )
    
    # Check accuracy
    print( sess.run( [hypothesis, tf.floor(hypothesis+0.5), correct_prediction, accuracy], 
                   feed_dict={X:x_data, Y:y_data}) )
    print( "Accuracy:", accuracy.eval({X:x_data, Y:y_data}) )

0 0.696061 [[ 0.77456301 -0.48697069 -0.79885447 -0.5950067  -0.24585538 -0.11151248
   0.96470636  0.59188402  0.46099636 -0.52361101]
 [ 0.88131416  0.35992622 -0.79917061  0.06534496  0.47518432 -0.09021433
   0.37483916 -0.17001189 -0.24022606 -0.67154294]] [[-0.04022731]
 [ 0.15905026]
 [-0.48526028]
 [-0.00326672]
 [ 0.21676567]
 [ 0.85250682]
 [ 0.20781513]
 [-0.3516489 ]
 [ 0.30496633]
 [-0.78694308]]
1000 0.651832 [[ 0.77410066 -0.55577266 -1.57609379 -0.61037874 -0.28601658 -0.27588239
   1.01678634  0.77786857  0.35235262 -1.29183412]
 [ 0.88063914  0.29358286 -1.53386986  0.05483     0.41053838 -0.37663597
   0.49211732 -0.09047327 -0.37439167 -1.29582608]] [[ 0.0361503 ]
 [ 0.17718995]
 [-1.23789036]
 [ 0.13720672]
 [ 0.06801979]
 [ 0.97272485]
 [ 0.13491561]
 [-0.56958252]
 [ 0.27930254]
 [-1.15142202]]
2000 0.483382 [[ 0.74046201 -0.81340808 -3.06410623 -0.82491833 -0.32115689 -0.56050038
   1.02751744  1.20046878  0.11870658 -2.55744076]
 [ 0.87130833  0.0759313  -2.982

## Use 'Deep' neural network to solve XOR problem.

In [58]:
xy = np.loadtxt('07train.txt', unpack=True)

x_data = np.transpose( xy[0:-1] )
y_data = np.reshape( xy[-1], (4,1) )

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# Deep network configuration.: Use more layers. 
W1 = tf.Variable(tf.random_uniform( [2,5], -1.0, 1.0)) # 3개의 층으로  in 2 out 5
W2 = tf.Variable(tf.random_uniform( [5,4], -1.0, 1.0)) # in 5 out 4 
W3 = tf.Variable(tf.random_uniform( [4,1], -1.0, 1.0)) # in 4 out 1


b1 = tf.Variable(tf.zeros([5]), name="Bias1") # 5개의 노드 Bias : 5개
b2 = tf.Variable(tf.zeros([4]), name="Bias2") # 4개의 노드 Bias : 4개
b3 = tf.Variable(tf.zeros([1]), name="Bias3") # 1개의 노드 Bias : 1개


# Hypotheses 
L2 =  tf.sigmoid(tf.matmul(X,W1)+b1)  # L2 = X * W1 + b1
L3 =  tf.sigmoid(tf.matmul(L2,W2)+b2) # L3 = L2 * W2 + b2
hypothesis = tf.sigmoid( tf.matmul(L3,W3) + b3) # hypothesis = L3 * W3 + b3

# Cost function 
cost = -tf.reduce_mean( Y*tf.log(hypothesis)+(1-Y)* tf.log(1.-hypothesis) )

# Minimize cost.
a = tf.Variable(0.1)
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

# Initializa all variables.
init = tf.initialize_all_variables()


# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(20001):
        sess.run(train, feed_dict={X:x_data, Y:y_data})
        
        if step % 1000 == 0:
            print(
                step, 
                sess.run(cost, feed_dict={X:x_data, Y:y_data}), 
                sess.run(W1),
                sess.run(W2)
            )
    
    # Test model
    correct_prediction = tf.equal( tf.floor(hypothesis+0.5), Y)
    accuracy = tf.reduce_mean(tf.cast( correct_prediction, "float" ) )
    
    # Check accuracy
    print( sess.run( [hypothesis, tf.floor(hypothesis+0.5), correct_prediction, accuracy], 
                   feed_dict={X:x_data, Y:y_data}) )
    print( "Accuracy:", accuracy.eval({X:x_data, Y:y_data}) )

0 0.701768 [[ 0.16835859 -0.92337316 -0.42157048  0.97754902 -0.06309025]
 [ 0.70236516 -0.43839204 -0.53003985 -0.42422858  0.92067373]] [[-0.68873584  0.60137308  0.16320528  0.39678356]
 [ 0.87818176  0.01208374 -0.88203037  0.92998332]
 [ 0.88057435 -0.84993207  0.96204716  0.31987014]
 [-0.77234721 -0.50768209  0.06943167  0.72321022]
 [ 0.73062056 -0.55624038 -0.81566727  0.09564757]]
1000 0.693411 [[ 0.11608291 -0.80341762 -0.45520112  0.92525971  0.12888812]
 [ 0.68962127 -0.24886326 -0.52172178 -0.41553217  0.92118776]] [[-0.68551153  0.57741421  0.15146606  0.41152474]
 [ 0.87660277  0.02597154 -0.82351452  0.92992872]
 [ 0.88262755 -0.84183574  0.99530303  0.32345551]
 [-0.76801842 -0.49198344  0.0596128   0.71994013]
 [ 0.73137307 -0.57562238 -0.81476873  0.10703145]]
2000 0.692283 [[ 0.07807797 -0.74109662 -0.50676763  0.87831134  0.38271081]
 [ 0.68336385 -0.14463721 -0.53097081 -0.42136103  0.99043131]] [[-0.68759042  0.57297879  0.15221986  0.41287977]
 [ 0.87675923  0.